# RNN from Scratch

Original post at <a rel="canonical" href="https://peterroelants.github.io/posts/rnn-implementation-part01/">peterroelants.github.io</a> is generated from an IPython notebook file. [Link to the full IPython notebook file](https://github.com/peterroelants/peterroelants.github.io/blob/master/notebooks/RNN_implementation/rnn-implementation-part01.ipynb)

This tutorial will illustrate how to implement a [simple RNN](#Linear-recurrent-neural-network) and how to train it with [Backpropagation through time](#Training-with-backpropagation-through-time) using [Rprop](#Rprop-optimisation) optimization.

## Recurrent neural networks

[Recurrent neural networks](https://en.wikipedia.org/wiki/Recurrent_neural_network) (RNNs) are neural nets that can deal with sequences of variable length (unlike feedforward nets). They are able to this by defining a [recurrence relation](https://en.wikipedia.org/wiki/Recurrence_relation) over timesteps which is typically the following formula:

$$
S_{k} = f(S_{k-1} \cdot W_{rec} + X_k \cdot W_x)
$$

Where $S_k$ is the state at time $k$, $X_k$ an input at time $k$, $W_{rec}$ and $W_x$ are parameters like the weights parameters in feedforward nets. Note that the RNN can be viewed as a state model with a [feedback loop](https://en.wikipedia.org/wiki/Feedback). 

The state evolves over time due to the recurrence relation, and the feedback is fed back into the state with a delay of one timestep. This delayed feedback loop gives the model memory because it can remember information between timesteps in the states.  
The final output of the network $Y_k$ at a certain timestep $k$ is typically computed from one or more states $S_{k-i} \cdots S_{k+j}$.

## Vanilla RNN

The first part of this tutorial describes a simple RNN that is trained to count how many 1's it sees on a binary input stream, and output the total count at the end of the sequence.

The RNN model used here has one state, takes one input element from the binary stream each timestep, and outputs its last state at the end of the sequence. This model is shown in the figure below. The left part is a graphical illustration of the recurrence relation it describes ($ s_{k} = s_{k-1} \cdot w_{rec} + x_k \cdot w_x $). The right part illustrates how the network is unfolded through time over a sequence of length n. Notice that the unfolded network can be viewed as an (n+1)-layer neural network with the same shared parameters $w_{rec}$ and $w_x$ in each layer.

![Structure of the linear RNN](https://peterroelants.github.io/images/RNN_implementation/SimpleRNN01.png)

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set the seed for reproducability
np.random.seed(seed=1)

%matplotlib inline

## Define the dataset

The input data $X$ used in this example consists of 20 binary sequences of 10 timesteps each. Each input sequence is generated from a uniform random distribution which is rounded to $0$ or $1$.

The output targets $t$ are the number of times '1' occurs in the sequence, which is equal to the sum of that sequence since the sequence is binary.

In [2]:
# Create dataset
nb_of_samples = 20
sequence_len = 10

# Create the sequences
X = np.zeros((nb_of_samples, sequence_len))
for row_idx in range(nb_of_samples):
    X[row_idx,:] = np.around(np.random.rand(sequence_len)).astype(int)

# Create the targets for each sequence
t = np.sum(X, axis=1)

In [3]:
X

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0., 1., 0., 1., 0., 0.],
       [1., 1., 0., 1., 1., 1., 0., 0., 0., 1.],
       [0., 0., 1., 1., 1., 0., 1., 1., 0., 1.],
       [1., 1., 0., 1., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 1., 1.],
       [0., 0., 1., 0., 0., 1., 1., 1., 1., 1.],
       [1., 0., 0., 1., 0., 0., 1., 0., 1., 1.],
       [1., 1., 1., 0., 0., 1., 0., 1., 1., 1.],
       [0., 1., 0., 1., 0., 0., 1., 1., 0., 1.],
       [0., 1., 1., 0., 1., 1., 0., 1., 1., 1.],
       [0., 0., 1., 1., 0., 1., 1., 1., 1., 0.],
       [0., 0., 0., 0., 1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 0., 1., 0., 1., 0., 1., 1.],
       [1., 0., 0., 0., 0., 0., 0., 1., 1., 0.],
       [0., 1., 1., 0., 0., 1., 0., 1., 1., 1.],
       [0., 0., 1., 1., 0., 0., 0., 0., 1., 1.],
       [0., 1., 1., 0., 1., 1., 1., 0., 0., 0.],
       [1., 0., 1., 0., 0., 1., 0., 1., 1., 1.],
       [0., 0., 1., 1., 1., 1., 0., 0., 1., 1.]])

In [4]:
t

array([2., 4., 6., 6., 4., 4., 6., 5., 7., 5., 7., 6., 4., 7., 3., 6., 4.,
       5., 6., 6.])

## Training with backpropagation through time

The typical algorithm to train recurrent nets is the [backpropagation through time](https://en.wikipedia.org/wiki/Backpropagation_through_time) algorithm. 

### Step 1: Compute the output with the forward step

The forward step will unroll the network, and compute the forward activations just as in regular backpropagation. The final output will be used to compute the loss function from which the error signal used for training will be derived.

When unfolding a recurrent net over multiple timesteps each layer computes the same recurrence relation on different timesteps. This recurrence relation for our model is defined in the `update_state` method.

The `forward_states` method computes the states over increasing timesteps by applying the `update_state` method in a for-loop. 

Since the network begins without seeing anything of the sequence, an initial state needs to be provided. In this example, this initial state is set to $0$ (it is possible to treat it as another parameter).

Finally, the loss $\xi$ (`loss`) at the output is computed in this example via the mean squared error function (MSE) over all sequences in the input data.

In [5]:
# Define the forward step functions

def update_state(xk, sk, wx, wRec):
    """
    Compute state k from the previous state (sk) and current 
    input (xk), by use of the input weights (wx) and recursive 
    weights (wRec).
    """
    return xk * wx + sk * wRec


def forward_states(X, wx, wRec):
    """
    Unfold the network and compute all state activations 
    given the input X, input weights (wx), and recursive weights 
    (wRec). Return the state activations in a matrix, the last 
    column S[:,-1] contains the final activations.
    """
    # Initialise the matrix that holds all states for all 
    #  input sequences. The initial state s0 is set to 0.

    S = np.zeros((X.shape[0], X.shape[1]+1))
    
    # Use the recurrence relation defined by update_state to update 
    #  the states trough time.
    for k in range(0, X.shape[1]):
        # S[k] = S[k-1] * wRec + X[k] * wx
        S[:,k+1] = update_state(X[:,k], S[:,k], wx, wRec)
    return S


def loss(y, t): 
    """MSE between the targets t and the outputs y."""
    return np.mean((t - y)**2)

### Step 2: Compute the gradients with the backward step

The backward step will begin with computing the gradient of the loss with respect to the output of the network $\partial \xi / \partial y$ by the `output_gradient` method. This gradient will then be propagated backwards through time (layer by layer) from output to input to update the parameters by the `backward_gradient` method. The recurrence relation to propagate this gradient through the network can be written as:

$$\frac{\partial \xi}{\partial S_{k-1}} 
= \frac{\partial \xi}{\partial S_{k}} \frac{\partial S_{k}}{\partial S_{k-1}}
= \frac{\partial \xi}{\partial S_{k}} w_{rec}$$

and starts at:

$$\frac{\partial \xi}{\partial y} = \frac{\partial \xi}{\partial S_{n}}$$

With $n$ the number of timesteps the netwoark is unfolded. Note that only the recursive parameter that connects states $w_{rec}$ plays a role in propagating the error down the network.

The gradients of the loss function with respect to the parameters can then be found by summing the parameter gradients in each layer (or accumulating them while propagating the error).

$$\frac{\partial \xi}{\partial w_x} 
= \sum_{k=0}^{n} \frac{\partial \xi}{\partial S_{k}} x_k
\\
\frac{\partial \xi}{\partial w_{rec}} 
= \sum_{k=1}^{n} \frac{\partial \xi}{\partial S_{k}} S_{k-1}$$

In [6]:
def output_gradient(y, t):
    """
    Gradient of the MSE loss function with respect to the output y.
    """
    return 2. * (y - t)


def backward_gradient(X, S, grad_out, wRec):
    """
    Backpropagate the gradient computed at the output (grad_out) 
    through the network. Accumulate the parameter gradients for 
    wX and wRec by for each layer by addition. Return the parameter 
    gradients as a tuple, and the gradients at the output of each layer.
    """
    # Initialise the array that stores the gradients of the loss with 
    #  respect to the states.
    grad_over_time = np.zeros((X.shape[0], X.shape[1]+1))
    grad_over_time[:,-1] = grad_out
    
    # Set the gradient accumulations to 0
    wx_grad = 0
    wRec_grad = 0

    for k in range(X.shape[1], 0, -1):
        # Compute the parameter gradients and accumulate the results.
        wx_grad += np.sum(
            np.mean(grad_over_time[:,k] * X[:,k-1], axis=0))
        wRec_grad += np.sum(
            np.mean(grad_over_time[:,k] * S[:,k-1]), axis=0)
        # Compute the gradient at the output of the previous layer
        grad_over_time[:,k-1] = grad_over_time[:,k] * wRec
    return (wx_grad, wRec_grad), grad_over_time

### Training: Rprop optimisation

The gradient of a Vanilla RNN can be very unstable. Entering an area of unstable gradients could result in a huge jump on the loss surface, where the optimizer might end up far from the original point. This is illustrated in the following figure (from [Pascanu et al.](http://www.jmlr.org/proceedings/papers/v28/pascanu13.pdf)):

![Illustration exploding gradient from "On the difficulty of training Recurrent Neural Networks", Razvan Pascanu](https://peterroelants.github.io/images/RNN_implementation/ExplodingGradient_Razvan.png)

Remember that the gradient descent update rule was:

$$
W(i+1) = W(i) - \mu \frac{\partial \xi}{\partial W(i)}
$$

With $W(i)$ the value of $W$ at iteration $i$ during the gradient descent, and $\mu$ the learning rate. 

If during training we would end up in the blue point in the surface plot above above ($w_x\!=\!1, w_{rec}\!=\!2$) the gradient would be in the order of $10^7$. Even with a small learning rate of $0.000001$ ($10^{-6}$) the parameters $W$ would be updated a distance 10 units from its current position, which would be catastrophic in our example. One way do deal with this is to lower the learning rate even more, but if then the optimisation enters a low gradient area the updates wouldn't move at all.

Researchers have found many ways to do gradient based training in this unstable environment. Some examples are: [Gradient clipping](http://arxiv.org/pdf/1211.5063v2.pdf), [Hessian-Free Optimization](http://www.icml-2011.org/papers/532_icmlpaper.pdf), [Momentum](http://www.cs.utoronto.ca/~ilya/pubs/ilya_sutskever_phd_thesis.pdf), etc.

We can handle the unstable gradients by making the optimisation updates less sensitive to the gradients. One way to do this is by using a technique called [resilient backpropagation (Rprop)](https://en.wikipedia.org/wiki/Rprop). Rprop uses the sign of the gradient to determine the direction of update.

In [7]:
# Define Rprop optimisation function
def update_rprop(X, t, W, W_prev_sign, W_delta, eta_p, eta_n):
    """
    Update Rprop values in one iteration.
    Args:
        X: input data.
        t: targets.
        W: Current weight parameters.
        W_prev_sign: Previous sign of the W gradient.
        W_delta: Rprop update values (Delta).
        eta_p, eta_n: Rprop hyperparameters.
    Returns:
        (W_delta, W_sign): Weight update and sign of last weight
                           gradient.
    """
    # Perform forward and backward pass to get the gradients
    S = forward_states(X, W[0], W[1])
    grad_out = output_gradient(S[:,-1], t)
    W_grads, _ = backward_gradient(X, S, grad_out, W[1])
    W_sign = np.sign(W_grads)  # Sign of new gradient

    # Update the Delta (update value) for each weight 
    #  parameter seperately
    for i, _ in enumerate(W):
        if W_sign[i] == W_prev_sign[i]:
            W_delta[i] *= eta_p
        else:
            W_delta[i] *= eta_n
    return W_delta, W_sign

In [8]:
# Perform Rprop optimisation

# Set hyperparameters
eta_p = 1.2
eta_n = 0.5

# Set initial parameters
W = [-1.5, 2]  # [wx, wRec]
W_delta = [0.001, 0.001]  # Update values (Delta) for W
W_sign = [0, 0]  # Previous sign of W

ls_of_ws = [(W[0], W[1])]  # List of weights to plot
# Iterate over 500 iterations
for i in range(500):
    # Get the update values and sign of the last gradient
    W_delta, W_sign = update_rprop(
        X, t, W, W_sign, W_delta, eta_p, eta_n)
    # Update each weight parameter seperately
    for i, _ in enumerate(W):
        W[i] -= W_sign[i] * W_delta[i]
    ls_of_ws.append((W[0], W[1]))  # Add weights to list to plot

print(f'Final weights are: wx = {W[0]:.4f},  wRec = {W[1]:.4f}')

Final weights are: wx = 1.0014,  wRec = 0.9997


## Final model

The final model is tested on a test sequence below. Note that the output is very close to the target output and that if we round the output to the nearest integer that the output would be perfect.

In [9]:
test_inpt = np.asmatrix([[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1]])
test_outpt = forward_states(test_inpt, W[0], W[1])[:,-1]
sum_test_inpt = test_inpt.sum()
print((
    f'Target output: {sum_test_inpt:d} vs Model output: '
    f'{test_outpt[0]:.2f}'))
#

Target output: 5 vs Model output: 5.00


# Ready for more?

LSTM implemented in numpy: https://github.com/nicodjimenez/lstm